In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean, DateTime, ForeignKey, Float, Time, Table
import psycopg2
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
from psycopg2 import extensions
from datetime import datetime
import os
from moviepy.editor import VideoFileClip
import pandas as pd

In [2]:
Base = declarative_base()

class Users(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    surename = Column(String)

    loops = relationship('Loop', back_populates='user')

#Relation Tables for M:M
tools_loop_association = Table(
    'tools_loop_association',
    Base.metadata,
    Column('tool_id', Integer, ForeignKey('tools.id')),
    Column('loop_id', Integer, ForeignKey('loop.id'))
)

bodyside_loop_association = Table(
    'bodyside_loop_association',
    Base.metadata,
    Column('bodyside_id', Integer, ForeignKey('bodyside.id')),
    Column('loop_id', Integer, ForeignKey('loop.id'))
)

class Tools(Base):
    __tablename__ = 'tools'

    id = Column(Integer, primary_key=True)
    tool = Column(String)
    time_in_use = Column(Time)

    #M:M
    loop = relationship('Loop', secondary=tools_loop_association, back_populates='tools')

class Bodyside(Base):
    __tablename__ = 'bodyside'

    id = Column(Integer, primary_key=True)
    side = Column(String)
    time_in_use = Column(Time)

    #M:M
    loop = relationship('Loop', secondary=bodyside_loop_association, back_populates='bodysides')


class Loop(Base):
    __tablename__ = 'loop'

    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime)
    session_length = Column(Integer) #Seconds
    location = Column(String)
    human_labeled = Column(Boolean)

    #1:M
    videos = relationship('Video', back_populates='loop')

    #M:1
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('Users', back_populates='loops')

    #M:M
    tools = relationship('Tools', secondary=tools_loop_association, back_populates='loop')
    bodysides = relationship('Bodyside', secondary=bodyside_loop_association, back_populates='loop')


class Video(Base):
    __tablename__= 'video'

    id = Column(Integer, primary_key=True)
    device = Column(String)
    file_path = Column(String)

    #M:1
    loop_id = Column(Integer, ForeignKey('loop.id'))
    loop = relationship('Loop', back_populates='videos')

In [3]:
db_params = {
    'user': 'postgres',
    'password': 'Digilarger',  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'bigblock', #tensionTerminator
}

print(f"This notebook will create a new Database with the name: {db_params['database']}\n")

connection = psycopg2.connect(
    user=db_params['user'],
    password=db_params['password'],
    host=db_params['host'],
    port=db_params['port'],
)
print(f"{connection}\n")

db_url = f"postgresql://" \
            f"{db_params['user']}:" \
            f"{db_params['password']}@" \
            f"{db_params['host']}:" \
            f"{db_params['port']}/" \
            f"{db_params['database']}"

engine = create_engine(db_url, echo=True)
Session = sessionmaker(bind=engine)
session = Session()
print(f"Connected to PostgreSQL, DB: {db_params['database']}")

This notebook will create a new Database with the name: bigblock

<connection object at 0x00000240E157B010; dsn: 'user=postgres password=xxx host=localhost port=5432', closed: 0>

Connected to PostgreSQL, DB: bigblock


In [4]:
user = session.query(Users).filter_by(id=2).first()
print(user.name, user.surename)

2023-10-01 09:10:36,487 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-10-01 09:10:36,488 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 09:10:36,489 INFO sqlalchemy.engine.Engine select current_schema()
2023-10-01 09:10:36,490 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 09:10:36,492 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-10-01 09:10:36,492 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 09:10:36,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 09:10:36,496 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.surename AS users_surename 
FROM users 
WHERE users.id = %(id_1)s 
 LIMIT %(param_1)s
2023-10-01 09:10:36,497 INFO sqlalchemy.engine.Engine [generated in 0.00106s] {'id_1': 2, 'param_1': 1}
Christina Greiderer


In [5]:
loop = session.query(Loop).filter_by(human_labeled=True).all()

2023-10-01 09:10:36,513 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop 
WHERE loop.human_labeled = true
2023-10-01 09:10:36,513 INFO sqlalchemy.engine.Engine [generated in 0.00060s] {}


In [6]:
loops_with_trigger = session.query(Loop).join(tools_loop_association).filter(
    Loop.human_labeled == True,
    tools_loop_association.c.tool_id == 1  # Modify this condition as needed
).all()

2023-10-01 09:10:36,527 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop JOIN tools_loop_association ON loop.id = tools_loop_association.loop_id 
WHERE loop.human_labeled = true AND tools_loop_association.tool_id = %(tool_id_1)s
2023-10-01 09:10:36,527 INFO sqlalchemy.engine.Engine [generated in 0.00066s] {'tool_id_1': 1}


In [7]:
loops_with_buoballs = session.query(Loop).join(tools_loop_association).filter(
    Loop.human_labeled == True,
    tools_loop_association.c.tool_id != 1  # Modify this condition as needed
).all()

2023-10-01 09:10:36,544 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop JOIN tools_loop_association ON loop.id = tools_loop_association.loop_id 
WHERE loop.human_labeled = true AND tools_loop_association.tool_id != %(tool_id_1)s
2023-10-01 09:10:36,544 INFO sqlalchemy.engine.Engine [generated in 0.00065s] {'tool_id_1': 1}


In [8]:
loops_with_trigger_ids = []
loops_with_buoballs_ids = []

for x in loops_with_trigger:
    loops_with_trigger_ids.append(x.id)

for y in loops_with_buoballs:
    loops_with_buoballs_ids.append(y.id)

In [9]:
rgb_data_with_trigger = session.query(Video).join(Loop, Video.loop_id == Loop.id).filter(
    Video.device == 'rgbCam',
    Loop.id.in_(loops_with_trigger_ids)
).all()

2023-10-01 09:10:36,576 INFO sqlalchemy.engine.Engine SELECT video.id AS video_id, video.device AS video_device, video.file_path AS video_file_path, video.loop_id AS video_loop_id 
FROM video JOIN loop ON video.loop_id = loop.id 
WHERE video.device = %(device_1)s AND loop.id IN (%(id_1_1)s, %(id_1_2)s, %(id_1_3)s, %(id_1_4)s, %(id_1_5)s, %(id_1_6)s, %(id_1_7)s, %(id_1_8)s, %(id_1_9)s, %(id_1_10)s, %(id_1_11)s, %(id_1_12)s, %(id_1_13)s, %(id_1_14)s, %(id_1_15)s, %(id_1_16)s, %(id_1_17)s, %(id_1_18)s, %(id_1_19)s, %(id_1_20)s, %(id_1_21)s, %(id_1_22)s, %(id_1_23)s, %(id_1_24)s, %(id_1_25)s, %(id_1_26)s, %(id_1_27)s, %(id_1_28)s, %(id_1_29)s, %(id_1_30)s, %(id_1_31)s, %(id_1_32)s, %(id_1_33)s, %(id_1_34)s, %(id_1_35)s, %(id_1_36)s, %(id_1_37)s, %(id_1_38)s, %(id_1_39)s, %(id_1_40)s, %(id_1_41)s, %(id_1_42)s, %(id_1_43)s, %(id_1_44)s, %(id_1_45)s, %(id_1_46)s, %(id_1_47)s, %(id_1_48)s, %(id_1_49)s, %(id_1_50)s, %(id_1_51)s, %(id_1_52)s, %(id_1_53)s, %(id_1_54)s, %(id_1_55)s, %(id_1_56)s, %

In [10]:
rgb_data_with_buoballs = session.query(Video).join(Loop, Video.loop_id == Loop.id).filter(
    Video.device == 'rgbCam',
    Loop.id.in_(loops_with_buoballs_ids)
).all()

2023-10-01 09:10:36,591 INFO sqlalchemy.engine.Engine SELECT video.id AS video_id, video.device AS video_device, video.file_path AS video_file_path, video.loop_id AS video_loop_id 
FROM video JOIN loop ON video.loop_id = loop.id 
WHERE video.device = %(device_1)s AND loop.id IN (%(id_1_1)s, %(id_1_2)s, %(id_1_3)s, %(id_1_4)s, %(id_1_5)s, %(id_1_6)s, %(id_1_7)s, %(id_1_8)s, %(id_1_9)s, %(id_1_10)s, %(id_1_11)s, %(id_1_12)s, %(id_1_13)s, %(id_1_14)s, %(id_1_15)s, %(id_1_16)s, %(id_1_17)s, %(id_1_18)s, %(id_1_19)s, %(id_1_20)s, %(id_1_21)s, %(id_1_22)s, %(id_1_23)s, %(id_1_24)s, %(id_1_25)s, %(id_1_26)s, %(id_1_27)s, %(id_1_28)s, %(id_1_29)s, %(id_1_30)s, %(id_1_31)s, %(id_1_32)s, %(id_1_33)s, %(id_1_34)s, %(id_1_35)s, %(id_1_36)s, %(id_1_37)s, %(id_1_38)s, %(id_1_39)s, %(id_1_40)s, %(id_1_41)s, %(id_1_42)s, %(id_1_43)s, %(id_1_44)s, %(id_1_45)s, %(id_1_46)s, %(id_1_47)s, %(id_1_48)s, %(id_1_49)s, %(id_1_50)s, %(id_1_51)s, %(id_1_52)s, %(id_1_53)s, %(id_1_54)s)
2023-10-01 09:10:36,592 INF

In [11]:
len(rgb_data_with_trigger)

74

In [12]:
len(rgb_data_with_buoballs)

54

In [14]:
trigger_file_path_list = []
buoballs_file_path_list = []

for x in rgb_data_with_trigger:
    trigger_file_path_list.append(x.file_path)

for x in rgb_data_with_buoballs:
    buoballs_file_path_list.append(x.file_path)

In [15]:
trigger_file_path_list

['D:/tt_video_data/videos-2023-09-08 19_12_12.778376/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_15_23.051845/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_20_38.981489/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_21_18.026639/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_23_23.087926/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_26_59.787451/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_27_26.236903/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_29_42.842763/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_30_59.265265/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_31_24.806689/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_33_59.031174/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_35_40.898982/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_37_26.155858/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-08 19_39_06.519723/rgbCam.h265.mp4',
 'D:/t

In [16]:
buoballs_file_path_list

['D:/tt_video_data/videos-2023-09-18 13_23_20.718099/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_24_13.346946/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_26_02.170698/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_26_50.579833/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_35_56.665269/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_37_12.229218/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_38_23.01515/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_39_49.382423/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_41_09.870865/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_44_29.648522/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_48_23.247632/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_49_18.154482/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_50_50.445568/rgbCam.h265.mp4',
 'D:/tt_video_data/videos-2023-09-18 13_52_10.52593/rgbCam.h265.mp4',
 'D:/tt_